# Korean-Abstractive-Summarization: huggingface_t5-large-korean-text-summary 
- TEAM: ChatGPT

## 1. Install and Setting library 

In [1]:
# huggingface transformers library
!pip install transformers
!pip install --upgrade --pre transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install peft 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 27.7 MB/s eta 0:00:00


In [3]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# dataset library
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.4 MB/s eta 0:00:00


In [5]:
!pip uninstall sentencepiece
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.2 MB/s eta 0:00:00


### 2-3. Split train and test dataset 

In [6]:
# 구글 드라이브와 연동합니다. 권한 허용이 필요합니다.
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [7]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/공모전/train_final.csv')

In [8]:
from sklearn.model_selection import train_test_split

# train과 test 데이터로 분리
train_df, test_df = train_test_split(train, test_size=0.2, random_state=24)


## 3. Load model and tokenizer
- Train Model: T5
    - https://huggingface.co/noahkim/KoT5_news_summarization



In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer

- fine tuning을 위한 tokenizer 및 model 로드

In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5Tokenizer

#tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/공모전/results_t5/checkpoint-900")
#model = AutoModelForSeq2SeqLM.from_pretrained("lcw99/t5-base-korean-text-summary") 

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/공모전/results_t5/checkpoint-900")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/공모전/results_t5/checkpoint-900") # 

Some weights of the model checkpoint at /content/drive/MyDrive/공모전/results_t5 were not used when initializing T5ForConditionalGeneration: ['encoder.block.8.layer.0.SelfAttention.q.lora_A.default.weight', 'encoder.block.9.layer.0.SelfAttention.v.lora_B.default.weight', 'encoder.block.4.layer.0.SelfAttention.q.lora_A.default.weight', 'encoder.block.6.layer.0.SelfAttention.q.lora_B.default.weight', 'decoder.block.10.layer.0.SelfAttention.v.lora_A.default.weight', 'encoder.block.3.layer.0.SelfAttention.v.lora_A.default.weight', 'decoder.block.11.layer.0.SelfAttention.v.lora_A.default.weight', 'encoder.block.2.layer.0.SelfAttention.v.lora_A.default.weight', 'decoder.block.7.layer.0.SelfAttention.v.lora_A.default.weight', 'decoder.block.0.layer.0.SelfAttention.v.lora_B.default.weight', 'decoder.block.10.layer.0.SelfAttention.q.lora_B.default.weight', 'decoder.block.0.layer.1.EncDecAttention.q.lora_B.default.weight', 'decoder.block.2.layer.0.SelfAttention.q.lora_B.default.weight', 'decoder.bl

- device 설정 (GPU 사용을 권장)

In [14]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

- 입력과 출력 길이 설정

In [15]:
max_input_length = 1024
max_output_length = 102

- 입력 및 출력 토큰화 함수 정의

In [16]:
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=max_input_length)

def summarize(batch):
    return tokenizer(batch['summary'], padding='max_length', truncation=True, max_length=max_output_length)

- 학습 데이터셋 및 데이터로더 생성

In [17]:
train_dataset = []
for i in range(len(train_df)):
    data = {
        'id': train_df.iloc[i]['id'],
        'input_ids': None,
        'attention_mask': None,
        'decoder_input_ids': None,
        'decoder_attention_mask': None,
        'labels': None
    }

    # 입력과 출력을 토큰화
    input = tokenizer.encode_plus(train_df.iloc[i]['text'], max_length=max_input_length, padding='max_length', truncation=True, return_tensors='pt')
    output = tokenizer.encode_plus(train_df.iloc[i]['summary'], max_length=max_output_length, padding='max_length', truncation=True, return_tensors='pt')

    # 토큰 ID와 어텐션 마스크를 입력 데이터로 저장
    data['input_ids'] = input['input_ids'][0]
    data['attention_mask'] = input['attention_mask'][0]

    # 토큰 ID와 어텐션 마스크를 출력 데이터로 저장
    data['decoder_input_ids'] = output['input_ids'][0]
    data['decoder_attention_mask'] = output['attention_mask'][0]

    # 라벨 데이터로 저장
    data['labels'] = output['input_ids'][0].clone().detach()

    # 라벨에서 PAD 토큰의 위치를 마스크하여 불필요한 손실 계산 방지
    data['labels'][data['labels'] == tokenizer.pad_token_id] = -100

    train_dataset.append(data)

- 테스트 데이터셋 및 데이터로더 생성

In [18]:
test_dataset = []
for i in range(len(test_df)):
    data = {
        'id': test_df.iloc[i]['id'],
        'input_ids': None,
        'attention_mask': None,
        'decoder_input_ids': None,
        'decoder_attention_mask': None,
        'labels': None
    }

    # 입력과 출력을 토큰화
    input = tokenizer.encode_plus(test_df.iloc[i]['text'], max_length=max_input_length, padding='max_length', truncation=True, return_tensors='pt')
    output = tokenizer.encode_plus(test_df.iloc[i]['summary'], max_length=max_output_length, padding='max_length', truncation=True, return_tensors='pt')

    # 토큰 ID와 어텐션 마스크를 입력 데이터로 저장
    data['input_ids'] = input['input_ids'][0]
    data['attention_mask'] = input['attention_mask'][0]

    # 토큰 ID와 어텐션 마스크를 출력 데이터로 저장
    data['decoder_input_ids'] = output['input_ids'][0]
    data['decoder_attention_mask'] = output['attention_mask'][0]

    # 라벨 데이터로 저장
    data['labels'] = output['input_ids'][0].clone().detach()

    # 라벨에서 PAD 토큰의 위치를 마스크하여 불필요한 손실 계산 방지
    data['labels'][data['labels'] == tokenizer.pad_token_id] = -100

    test_dataset.append(data)

- 데이터 수집기 및 학습/테스트 데이터셋 로더 생성

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,model=model)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=data_collator)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=True, collate_fn=data_collator)

# PEFT

In [20]:
from peft import get_peft_model, PeftModel, TaskType, LoraConfig

PEFT_CONFIG = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)
PEFT_MODEL = get_peft_model(model, PEFT_CONFIG)
PEFT_MODEL.to('cuda')

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(50358, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(50358, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(
                    in_features=768, out_features=768, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedding_B): Pa

## 5. Train model

In [21]:
from transformers import EarlyStoppingCallback

# EarlyStoppingCallback 초기화
early_stopping = EarlyStoppingCallback(early_stopping_patience=100, early_stopping_threshold=0.01)

In [22]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AdamW

# 파인튜닝을 위한 argument 설정
training_args = Seq2SeqTrainingArguments(
    output_dir='/content/drive/MyDrive/공모전/results_t5',
    evaluation_strategy = "steps",
    eval_steps = 100, # Number of update steps between two evaluations.
    save_steps= 100, # after # steps model is saved 
    warmup_steps= 200,# number of warmup steps for learning rate scheduler
    num_train_epochs = 5,
    learning_rate = 2e-5,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    weight_decay = 0.01,
    push_to_hub=False,
    logging_dir='/content/drive/MyDrive/공모전/logs_t5',
    logging_steps=100,
    overwrite_output_dir=True,
    predict_with_generate=True,
    load_best_model_at_end = True,
    gradient_accumulation_steps=2,
    seed=42
)
optimizer = AdamW(model.parameters(), lr=training_args.learning_rate, eps=1e-8)

# trainer 객체 생성
trainer = Seq2SeqTrainer(
    model=PEFT_MODEL,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[early_stopping],
    optimizers=(optimizer, None)
)

# 파인튜닝 시작
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
100,1.009500,0.742257
200,0.848400,0.723271
300,0.801700,0.709641
400,0.773200,0.701328
500,0.720100,0.693906
600,0.769600,0.680251
700,0.729500,0.672478
800,0.736900,0.665179
900,0.721200,0.662521


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 40>:40                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1664 in train                    │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args=args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1667 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1945 in _inner_training_loop     │
│                                                                                                  │
│   1942 │   │   │   │   if (                                                                      │
│   1943 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│   1944 │   │   │   │   │   and not is_torch_tpu_available()                                      │
│ ❱ 1945 │   │   │   │   │   and (torch.isnan(tr_loss_step) or torch.isinf(tr_loss_step))          │
│   1946 │   │   │   │   ):                                                                        │
│   1947 │   │   │   │   │   # if loss is nan or inf simply add the average of previous logged lo  │
│   1948 │   │   │   │   │   tr_loss += tr_loss / (1 + self.state.global_step - self._globalstep_  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [23]:
model.save_pretrained("/content/drive/MyDrive/공모전/results_t5") # 모델 저장장

In [24]:
trainer.evaluate()

Step,Training Loss,Validation Loss
100,1.009500,0.742257
200,0.848400,0.723271
300,0.801700,0.709641
400,0.773200,0.701328
500,0.720100,0.693906
600,0.769600,0.680251
700,0.729500,0.672478
800,0.736900,0.665179
900,0.721200,0.662521
918,0.721200,0.659639


{'eval_loss': 0.6596390604972839}